In [6]:
from __future__ import absolute_import
from __future__  import division
from __future__ import print_function
import tensorflow as tf
import numpy as np
from skimage.io import imread
from skimage.transform import resize
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [7]:
import os

In [8]:
from localfiletesting import *

In [9]:
model = mamon_videoFightModel2(tf,wight='modelweight.hdfs')

In [10]:
from tkinter import filedialog as fd
import tkinter
root = tkinter.Tk()
root.withdraw()
V_test = fd.askopenfilename()
cap = cv2.VideoCapture(V_test)
i = 0
frames = np.zeros((30, 160, 160, 3), dtype=float)
old = []
j = 0
truecount = 0
imagesaved=0
filename = 'savedImage.jpg'
ysdatav2 = np.zeros((1, 30, 160, 160, 3), dtype=float)
prediction = pred_fight(model,ysdatav2,acuracy=0.96)
while(True):
    ret, frame = cap.read()
  
    # describe the type of font
    # to be used.
    font = cv2.FONT_HERSHEY_SIMPLEX
    text_color = (0, 255, 0) #Green
    label = prediction[0]
    if label: # Violence prob
        text_color = (0, 0, 255) # red
        truecount = truecount + 1
    else:
        text_color = (0, 255, 0)
    text = "Violence: {}".format(label)
    cv2.putText(frame, text, (35, 50), font,1.25, text_color, 3)
    if i > 29:
        ysdatav2 = np.zeros((1, 30, 160, 160, 3), dtype=float)
        ysdatav2[0][:][:] = frames
        prediction = pred_fight(model,ysdatav2,acuracy=0.96)
        if prediction[0] == True:
            cv2.imshow('video', frame)
            print('Violence detected here ...')
            fourcc = cv2.VideoWriter_fourcc(*'XVID')
            vio = cv2.VideoWriter("./videos/output-"+str(j)+".avi", fourcc, 10.0, (fwidth,fheight))
            #vio = cv2.VideoWrite"./videos/output-"+str(j)+".mp4", cv2.VideoWriter_fourcc(*'mp4v'), 10, (300, 400))
            for frameinss in old:
                vio.write(frameinss)
            vio.release()
        i = 0
        j += 1
        frames = np.zeros((30, 160, 160, 3), dtype=float)
        old = []
    else:
        try:
            frm = resize(frame,(160,160,3))
            old.append(frame)
            fshape = frame.shape
            fheight = fshape[0]
            fwidth = fshape[1]
            frm = np.expand_dims(frm,axis=0)
            if(np.max(frm)>1):
                frm = frm/255
            frames[i][:] = frm
        
            i+=1
            cv2.imshow('video', frame)
        except:
            pass
            if cv2.waitKey(1):
                break
        
    if(truecount == 40):
        if(imagesaved == 0):
            if(label):
                cv2.imwrite(filename,frame)
                imagesaved = 1 
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()

cv2.destroyAllWindows()


1/1 [==============================] - 0s 47ms/step
